In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git


#All these packages need to be installed from pip
#These are all for the cluster detection
import sklearn
import sklearn.feature_extraction.text
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.datasets
import sklearn.cluster
import sklearn.decomposition
import sklearn.metrics

import scipy #For hierarchical clustering and some visuals
#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics
import seaborn as sns #Makes the graphics look nicer

# comp-linguistics
import spacy

#Displays the graphs
import graphviz

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

import itertools
import json
import re
import zipfile
import os
import sys
import pandas as pd
import random 

## Getting My Corpus

I am using the COCA corpus, which I have downloaded to my local computer.

In [4]:
corpus_name = "/Users/vedikaahuja/winter2020/Content-Analysis-2020/COCA"

In [5]:
def loadcorpus(corpus_name, corpus_style="text"):
    texts_raw = {}
    for file in os.listdir(corpus_name + "/"):
        if corpus_style in file:
            print(file)
            zfile = zipfile.ZipFile(corpus_name + "/" + file)
            for file in zfile.namelist():
                texts_raw[file] = []
                with zfile.open(file) as f:
                    for line in f:
                        texts_raw[file].append(line)
    return texts_raw

In [6]:
coca_raw = loadcorpus(corpus_name)

text_magazine_jex.zip
text_spoken_kde.zip
text_fiction_awq.zip
text_newspaper_lsp.zip
text_academic_rpe.zip
text_2012-2015_ksr.zip


In [ ]:
#sample_texts = random.sample(list(coca_raw.keys()), 15)

In [14]:
def clean_raw_text(raw_texts):
    clean_texts = []
    for text in raw_texts:
        try:
            text = text.decode("utf-8")
            clean_text = text.replace(" \'m", "'m").replace(" \'ll", "'ll").replace(" \'re", "'re").replace("\'s", "'s").replace(" \'re", "'re").replace(" n\'t", "n't").replace(" \'ve", "'ve").replace(" /'d", "'d").replace("<p>", "").replace("@", "")
            clean_texts.append(clean_text)
        except AttributeError:
            # print("ERROR CLEANING")
            # print(text)
            continue
        except UnicodeDecodeError:
            # print("Unicode Error, Skip")
            continue
    return clean_texts

In [16]:
text_df = {"year":[] , "genre":[], "text":[], "tokenized_text": []}

In [17]:
coca_texts = {}
for files in coca_raw:
    # collect year and genre info
    print(files)
    if files[0] != "w":
        year, genretxt = files.split("_")
        genre = genretxt[0:-4]
    else:
        w, genre, yeartxt = files.split("_")
        year = yeartxt[:4]
    coca_text = clean_raw_text(coca_raw[files][1:])
    for text_list in coca_text:
        txts = lucem_illud_2020.word_tokenize(text_list)
        try:
            coca_texts[txts[0][2:]] = txts[1:]
            
            #append info to dataframe
            text_df['year'].append(year)
            text_df['genre'].append(genre)
            text_df['text'].append(text_list)
            text_df['tokenized_text'].append(txts[1:])
            
        except IndexError:
            continue

w_mag_1990.txt
w_mag_1991.txt
w_mag_1992.txt
w_mag_1993.txt
w_mag_1994.txt
w_mag_1995.txt
w_mag_1997.txt
w_mag_1998.txt
w_mag_1999.txt
w_mag_2000.txt
w_mag_2001.txt
w_mag_2002.txt
w_mag_2003.txt
w_mag_2004.txt
w_mag_2005.txt
w_mag_2006.txt
w_mag_2007.txt
w_mag_2008.txt
w_mag_2009.txt
w_mag_2010.txt
w_mag_2011.txt
w_mag_2012.txt
w_mag_1996.txt
w_spok_1990.txt
w_spok_1991.txt
w_spok_1992.txt
w_spok_1993.txt
w_spok_1994.txt
w_spok_1995.txt
w_spok_1996.txt
w_spok_1997.txt
w_spok_1998.txt
w_spok_1999.txt
w_spok_2000.txt
w_spok_2001.txt
w_spok_2002.txt
w_spok_2003.txt
w_spok_2004.txt
w_spok_2005.txt
w_spok_2006.txt
w_spok_2007.txt
w_spok_2008.txt
w_spok_2009.txt
w_spok_2010.txt
w_spok_2011.txt
w_spok_2012.txt
w_fic_1990.txt
w_fic_1991.txt
w_fic_1992.txt
w_fic_1993.txt
w_fic_1994.txt
w_fic_1995.txt
w_fic_1996.txt
w_fic_1997.txt
w_fic_1998.txt
w_fic_1999.txt
w_fic_2000.txt
w_fic_2001.txt
w_fic_2002.txt
w_fic_2003.txt
w_fic_2004.txt
w_fic_2005.txt
w_fic_2006.txt
w_fic_2007.txt
w_fic_2008.txt
w_

In [20]:
coca_corpus = pd.DataFrame.from_dict(text_df)

In [ ]:
coca_corpus.to_pickle("full_corpora.pkl")

In [ ]:
coca_corpus.to_csv("1990_2011_coca_corpus.csv")

Create 5 year Epochs 

In [ ]:
coca_corpus['Epoch'] = .
coca_corpus['Epoch'] = np.where(coca_corpus['year'] < 1995, 1, 
                               np.where(coca_corpus['year'] < 2000, 2,
                                   np.where(coca_corpus['year'] < 2005, 3, 
                                           np.where(coca_corpus['year'] < 2010, 4,
                                                    np.where(coca_corpus['year'] < 2015, 5))))


In [2]:
coca_corpus = pd.read_csv("../week-6/1990_2011_coca_corpus.csv")

## Descriptive Statistics - Counting Words & Phrases (Week 4)

Count the number of times relevant words occur within the corpus

Normalize tokens? 

Explore the parts of speech associated with lesbian, gay, bisexual, transgender, queer and the adjectives associated with the nouns, in a small sample of the corpora from the early years (1990) compared to the later years (2012)

In [ ]:
def wordCounter(wordLst):
    wordCounts = {}
    for word in wordLst:
        #We usually need to normalize the case
        wLower = word.lower()
        if wLower in wordCounts:
            wordCounts[wLower] += 1
        else:
            wordCounts[wLower] = 1
    #convert to DataFrame
    countsForFrame = {'word' : [], 'count' : []}
    for w, c in wordCounts.items():
        countsForFrame['word'].append(w)
        countsForFrame['count'].append(c)
    return pandas.DataFrame(countsForFrame)

In [ ]:
# take a random sample of texts from 1990


## Vector Space Word Embeddings (Week 7)

In [ ]:
coca_corpus['tokenized_sents'] = coca_corpus['text'].apply(lambda x: [lucem_illud_2020.word_tokenize(s) for s in lucem_illud_2020.sent_tokenize(x)])
coca_corpus['normalized_sents'] = coca_corpus['tokenized_sents'].apply(lambda x: [lucem_illud_2020.normalizeTokens(s, lemma=False) for s in x])

In [ ]:
cocaW2V = gensim.models.word2vec.Word2Vec(coca_corpus['normalized_sents'].sum())

### Linguistic Change over Time (Week 7)

In [ ]:
import copy

In [ ]:
def calc_syn0norm(model):
    """since syn0norm is now depricated"""
    return (model.wv.syn0 / np.sqrt((model.wv.syn0 ** 2).sum(-1))[..., np.newaxis]).astype(np.float32)

def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    """Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
    (With help from William. Thank you!)
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.
    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """
    base_embed = copy.copy(base_embed)
    other_embed = copy.copy(other_embed)
    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)

    # get the embedding matrices
    base_vecs = calc_syn0norm(in_base_embed)
    other_vecs = calc_syn0norm(in_other_embed)

    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    # another matrix operation
    ortho = u.dot(v) 
    # Replace original array with modified one
    # i.e. multiplying the embedding matrix (syn0norm)by "ortho"
    other_embed.wv.syn0norm = other_embed.wv.syn0 = (calc_syn0norm(other_embed)).dot(ortho)
    return other_embed
    
def intersection_align_gensim(m1,m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.wv.vocab.keys())
    vocab_m2 = set(m2.wv.vocab.keys())

    # Find the common vocabulary
    common_vocab = vocab_m1&vocab_m2
    if words: common_vocab&=set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1-common_vocab and not vocab_m2-common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.vocab[w].count + m2.wv.vocab[w].count,reverse=True)

    # Then for each model...
    for m in [m1,m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.vocab[w].index for w in common_vocab]
        old_arr = calc_syn0norm(m)
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.syn0norm = m.wv.syn0 = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        m.index2word = common_vocab
        old_vocab = m.wv.vocab
        new_vocab = {}
        for new_index,word in enumerate(common_vocab):
            old_vocab_obj=old_vocab[word]
            new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
        m.wv.vocab = new_vocab

    return (m1,m2)

In [ ]:
def compareModels(df, category, text_column_name='normalized_sents', sort = True, embeddings_raw={}):
    """If you are using time as your category sorting is important"""
    if len(embeddings_raw) == 0:
        embeddings_raw = rawModels(df, category, text_column_name, sort)
    cats = sorted(set(df[category]))
    #These are much quicker
    embeddings_aligned = {}
    for catOuter in cats:
        embeddings_aligned[catOuter] = [embeddings_raw[catOuter]]
        for catInner in cats:
            embeddings_aligned[catOuter].append(smart_procrustes_align_gensim(embeddings_aligned[catOuter][-1], embeddings_raw[catInner]))
    return embeddings_raw, embeddings_aligned

In [ ]:
def rawModels(df, category, text_column_name='normalized_sents', sort = True):
    embeddings_raw = {}
    cats = sorted(set(df[category]))
    for cat in cats:
        #This can take a while
        print("Embedding {}".format(cat), end = '\r')
        subsetDF = df[df[category] == cat]
        #You might want to change the W2V parameters
        embeddings_raw[cat] = gensim.models.word2vec.Word2Vec(subsetDF[text_column_name].sum())
    return embeddings_raw

In [ ]:
#Generate Models
rawEmbeddings, comparedEmbeddings = compareModels(ascoDF, 'year')

In [ ]:
def getDivergenceDF(word, embeddingsDict):
    dists = []
    cats = sorted(set(embeddingsDict.keys()))
    dists = {}
    print(word)
    for cat in cats:
        dists[cat] = []
        for embed in embeddingsDict[cat][1:]:
            dists[cat].append(np.abs(1 - sklearn.metrics.pairwise.cosine_similarity(np.expand_dims(embeddingsDict[cat][0][word], axis = 0),
                                                                             np.expand_dims(embed[word], axis = 0))[0,0]))
    return pandas.DataFrame(dists, index = cats)

In [ ]:
targetWord = 'gay'

pltDF = getDivergenceDF(targetWord, comparedEmbeddings)
fig, ax = plt.subplots(figsize = (10, 7))
seaborn.heatmap(pltDF, ax = ax, annot = False) #set annot True for a lot more information
ax.set_xlabel("Starting year")
ax.set_ylabel("Final year")
ax.set_ylabel("Final year")
ax.set_title("Yearly linguistic change for: '{}'".format(targetWord))
plt.show()

In [ ]:
targetWord = 'queer'

pltDF = getDivergenceDF(targetWord, comparedEmbeddings)
fig, ax = plt.subplots(figsize = (10, 7))
seaborn.heatmap(pltDF, ax = ax, annot = False) #set annot True for a lot more information
ax.set_xlabel("Starting year")
ax.set_ylabel("Final year")
ax.set_ylabel("Final year")
ax.set_title("Yearly linguistic change for: '{}'".format(targetWord))
plt.show()

In [ ]:
targetWord = 'transgender'

pltDF = getDivergenceDF(targetWord, comparedEmbeddings)
fig, ax = plt.subplots(figsize = (10, 7))
seaborn.heatmap(pltDF, ax = ax, annot = False) #set annot True for a lot more information
ax.set_xlabel("Starting year")
ax.set_ylabel("Final year")
ax.set_ylabel("Final year")
ax.set_title("Yearly linguistic change for: '{}'".format(targetWord))
plt.show()

In [ ]:
targetWord = 'bisexual'

pltDF = getDivergenceDF(targetWord, comparedEmbeddings)
fig, ax = plt.subplots(figsize = (10, 7))
seaborn.heatmap(pltDF, ax = ax, annot = False) #set annot True for a lot more information
ax.set_xlabel("Starting year")
ax.set_ylabel("Final year")
ax.set_ylabel("Final year")
ax.set_title("Yearly linguistic change for: '{}'".format(targetWord))
plt.show()

In [ ]:
targetWord = 'lesbian'

pltDF = getDivergenceDF(targetWord, comparedEmbeddings)
fig, ax = plt.subplots(figsize = (10, 7))
seaborn.heatmap(pltDF, ax = ax, annot = False) #set annot True for a lot more information
ax.set_xlabel("Starting year")
ax.set_ylabel("Final year")
ax.set_ylabel("Final year")
ax.set_title("Yearly linguistic change for: '{}'".format(targetWord))
plt.show()

### Word Embeddings by Epoch

In [ ]:
rawEmbeddings_epoch = rawModels(coca_corpus, 'Epoch', text_column_name='normalized sents')    

In [ ]:
for epoch, embedding in enumerate(rawEmbeddings_epoch):
    model = rawEmbeddings_epoch[embedding]
    name = "embedding_epoch_" + str(epoch)
    model.save(name)

In [ ]:
from gensim.models import Word2Vec

In [ ]:
def file_to_embeddings(address, kind):
    rawEmbeddings = {}
    for file in os.listdir(address):
        if "embedding_"+kind in file:
            e, kind_, kind_type = file.split("_")
            kind_type = eval(kind_type)
            rawEmbeddings[kind_type] = Word2Vec.load(file)
    return rawEmbeddings

In [ ]:
rawEmbeddings_epoch_load = file_to_embeddings(".", "epoch")